In [1]:
from pyspark.sql import SparkSession
from random import randint
from pyspark.sql.types import IntegerType, StructField, StructType
import pyspark.sql.functions as F
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
import numpy as np

In [2]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable

In [3]:
spark = SparkSession.builder.appName('SparkTitanic.com').getOrCreate()

Reading the data

In [4]:
train_df = spark.read.option("inferSchema", True)\
    .option("header", True)\
    .csv('./input/titanic/train.csv')
    #for production: '../input/titanic/train.csv'

Cleaning the data

In [5]:
train_df = train_df.select("Sex", "Age", "Survived")

In [6]:
train_df = train_df.where(train_df["Age"].isNotNull())

In [7]:
train_df = train_df.where(train_df["Sex"].isNotNull())

In [8]:
train_df = train_df.where(train_df["Survived"].isNotNull())

In [9]:
train_df = train_df.withColumn("Sex", F.when(F.col("Sex")==F.lit("male"), 0.).otherwise(1.) )

In [10]:
x_train = list(map(lambda x: list(x), train_df.select("Sex", "Age").collect()))
y_train = list(map(lambda x: list(x)[0], train_df.select("Survived").collect()))


In [11]:
del(train_df)

In [12]:
x_train = np.array([[x[0], x[1]*0.01] for x in x_train], dtype="float32")
#normalization

In [13]:
y_train = to_categorical(y_train,2)

Training

In [14]:
model = Sequential()

In [15]:
model.add(Dense(128, input_dim = 2))

In [16]:
model.add(Dense(2084, activation='tanh'))

In [17]:
model.add(BatchNormalization(momentum=0.8))

In [18]:
model.add(Dense(128))

In [19]:
model.add(Dense(2084, activation='tanh'))

In [20]:
model.add(BatchNormalization(momentum=0.8))

In [21]:
model.add(Dense(128, activation='tanh'))

In [22]:
model.add(Dense(512, activation='tanh'))

In [23]:
model.add(BatchNormalization(momentum=0.8))

In [24]:
model.add(Dense(128, activation='tanh'))

In [25]:
model.add(Dense(2))

In [26]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [27]:
model.fit(x_train, y_train, batch_size=500, epochs=150)

Epoch 1/150
2/2 [==============================] - 1s 16ms/step - loss: 6.7814 - accuracy: 0.4048
Epoch 2/150
2/2 [==============================] - 0s 17ms/step - loss: 4.9596 - accuracy: 0.6933
Epoch 3/150
2/2 [==============================] - 0s 15ms/step - loss: 4.1378 - accuracy: 0.7003
Epoch 4/150
2/2 [==============================] - 0s 15ms/step - loss: 3.6740 - accuracy: 0.7437
Epoch 5/150
2/2 [==============================] - 0s 18ms/step - loss: 3.4244 - accuracy: 0.7759
Epoch 6/150
2/2 [==============================] - 0s 16ms/step - loss: 3.3724 - accuracy: 0.7801
Epoch 7/150
2/2 [==============================] - 0s 20ms/step - loss: 3.3724 - accuracy: 0.7801
Epoch 8/150
2/2 [==============================] - 0s 17ms/step - loss: 3.3724 - accuracy: 0.7801
Epoch 9/150
2/2 [==============================] - 0s 15ms/step - loss: 3.3724 - accuracy: 0.7801
Epoch 10/150
2/2 [==============================] - 0s 17ms/step - loss: 3.3724 - accuracy: 0.7801
Epoch 11/150
2/2 [=

In [28]:
del(x_train)
del(y_train)

Prediction

In [29]:
def prediction(row):
    if row.Sex == 'female':
        Sex = 1.
    elif row.Sex == 'male':
        Sex = 0.
    else:
        Sex = randint(0,1)*1.
    if row.Age == None:
        Age = randint(0,100)*0.01
    else:
        Age = row.Age*0.01

    p = model.predict([(Sex, Age)])[0]

    if p[0] > p[1]:
        return 0
    else:
        return 1

In [30]:
test_df = spark.read.option("inferSchema", True)\
    .option("header", True)\
    .csv('./input/titanic/test.csv')
    #for production: '../input/titanic/test.csv'

In [31]:
output_df = spark.createDataFrame(spark.sparkContext.parallelize([]), StructType([
    StructField('PassengerId', IntegerType(), True),
    StructField('Survived', IntegerType(), True)
]))

In [32]:
for row in test_df.collect():
    output_df = output_df.union(spark.sparkContext.parallelize([(row.PassengerId, prediction(row))]).toDF(['PassengerId', 'Survived']))

Output

In [33]:
output_df.toPandas().to_csv('./submission.csv', index=False)